In [18]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import json

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [12]:
with open ("data/vol7.json", "r", encoding="utf-8") as f:
    descs = json.load(f)["descriptions"]
with open ("data/vol7.json", "r", encoding="utf-8") as f:
    names = json.load(f)["names"]

In [3]:
# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [4]:
corpus_embeddings = model.encode(descs, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/340 [00:00<?, ?it/s]

In [ ]:
paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

In [6]:
paraphrases = util.paraphrase_mining(model, descs, corpus_chunk_size=len(descs), top_k=10)

In [9]:
print (len(paraphrases))

172422


In [17]:
for paraphrase in paraphrases:
    score, i, j = paraphrase
    if i == 0:
        sent1 = descs[i]
        sent2 = descs[j]
        print (f"Victim 1: Num: {names[i]}\n{sent1}\n")
        print (f"Victim 2: Num: {names[j]}\n{sent2}")
        print (f"{score}\n\n\n\n")

Victim 1: Num: AARON, Thabo Simon
An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.

Victim 2: Num: KGATLHANE, Tommy Kgosimang
An ANCYL member who was shot and injured by a member of the SAP in Griekwastad, Cape, on 2 July 1991, during conflict between the ANC and a vigilante group known as the Inkatha gang.
0.8138654828071594




Victim 1: Num: AARON, Thabo Simon
An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.

Victim 2: Num: GUZANA, Moses Phakamile
An ANCYL member who w

In [19]:
write_data("data/paraphrases-10.json", paraphrases)

In [ ]:
# clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

# print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# #Print for all clusters the top 3 and bottom 3 elements
# for i, cluster in enumerate(clusters):
#     print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
#     for sentence_id in cluster[0:3]:
#         print("\t", corpus_sentences[sentence_id])
#     print("\t", "...")
#     for sentence_id in cluster[-3:]:
#         print("\t", corpus_sentences[sentence_id])
